In [137]:
import numpy as np
import re
import pandas as pd

In [177]:
f = open('test3', 'r')
lines = f.readlines()

In [178]:
lines

['2*x1+x2-x3->min\n',
 '3*x1+2*x3>=12\n',
 'x1-x2+x3=5\n',
 'x1+x3<=6\n',
 'x1>=0\n',
 'x2>=0\n',
 'x3>=0']

In [179]:
#Составляем список и множество всех переменных, а также множество переменных, имеющих условие о положительности
var_list = re.findall(r"(?<=x)\d+", lines[0])
all_vars = set(var_list)
pat0 = re.compile(r"x\d+>=0\n*")
pos_vars = set()
pos_cond_line = 0
for i in range(1, len(lines)):
    if pat0.match(lines[i]):
        pos_vars.add(re.search(r'\d+', lines[i]).group())
        if pos_cond_line == 0:
            pos_cond_line = i

In [180]:
pos_vars

{'1', '2', '3'}

In [181]:
all_vars - pos_vars

set()

In [182]:
#Функция замены переменных, не имеющих условий положительности
def rand_var_replace(matr, c, var_to_replace):
    new_var_col = []
    for i in range(matr.shape[0]):
        new_var_col.append([-matr[i][var_to_replace - 1]])
    matr = np.hstack((matr, new_var_col))
    c = np.append(c, -c[var_to_replace - 1])
    return matr, c

In [188]:
#составляем матрицу A и вектора b и c. Также сохраним в переменную свободный коэффициент
#матрица A
A = ['' for _ in range(len(var_list))]

for j in range(1, pos_cond_line):
    #смотрим на условия построчно
    temp = lines[j]
    temp_arr = list()
    for k in var_list:
        #ищем переменную в строке и добавляем в таблицу коэффициент перед ней
        str_search0 = f"-?\d+(?=\*x{k})"
        coef = re.findall(str_search0, temp)
        if coef == []:
            str_search1 = f"-?x{k}"
            coef = re.findall(str_search1, temp)
            if coef == []:
                temp_arr.append(0)
            elif coef[0][0] == "-":
                temp_arr.append(-1)
            else:
                temp_arr.append(1)
        else:
            temp_arr.append(str(coef[0]))
    A = np.vstack([A, temp_arr])
A = np.delete(A, 0, axis = 0)
A = A.astype(np.float64)

#вектор b
b = np.array([])
for l in range(1, pos_cond_line):
    temp = re.findall(r"<=-?\d+|>=-?\d+", lines[l])
    if not temp:
        temp = re.findall(r"=-?\d+", lines[l])
    b = np.append(b, temp)

#вектор c
lines[0] = lines[0].replace('+x', '+1*x')
lines[0] = lines[0].replace('-x', '-1*x')
c = np.array(re.findall(r"-?\d+(?=\*x)", lines[0]))
с = c.astype(str)
c = c.astype(np.float64)

#свободный элемент Q
neg_Q = re.findall(r"[+,-]\d+(?=-)", lines[0])
if neg_Q:
    neg_Q = neg_Q[0]
    if neg_Q[0] == '+':
        neg_Q = '-' + neg_Q[1:]
    else:
        neg_Q = neg_Q[1:]
    neg_Q = float(neg_Q)
else:
    neg_Q = 0.0

In [189]:
#проверка, все ли переменные имеют условие о положительности, и добавление новых переменных с данным условием
neg_replacements = {}
if pos_vars != all_vars:
    neg_vars = all_vars - pos_vars
    for var in neg_vars:
        #создаём словарь, где ключом является номер заменяемой переменной (на его место помещается переменная из замены,
        #идущая с плюсом), а значением - номер, который для простоты присваивается переменной из замены, идущей с плюсом.
        neg_replacements[var] = A.shape[1] + 1
        A, c = rand_var_replace(A, c, int(var))

In [190]:
c

array([ 2.,  1., -1.])

In [191]:
A

array([[ 3.,  0.,  2.],
       [ 1., -1.,  1.],
       [ 1.,  0.,  1.]])

In [192]:
b

array(['>=12', '=5', '<=6'], dtype='<U32')

In [193]:
neg_Q

0.0

Приведение к каноническому виду

In [194]:
#Проверка того, что решается задача минимизации, приведение к ней при необходимости
if lines[0][-4:-1] == 'max':
    c *= -1
    neg_Q *= -1

In [195]:
#Устранение неравенств, приведение их к равенствам посредством добавления новых переменных
for i in range(len(b)):
    if "<" in b[i]:
        b[i] = re.findall(r"-?\d+", b[i])[0]
        A = np.hstack((A, np.zeros((3, 1), dtype="float")))
        A[i][-1] = 1.0
        c = np.append(c, 0.0)
    elif ">" in b[i]:
        b[i] = re.findall(r"-?\d+", b[i])[0]
        A = np.hstack((A, np.zeros((3, 1), dtype="float")))
        A[i][-1] = -1.0
        c = np.append(c, 0.0)
    else:
        b[i] = re.findall(r"-?\d+", b[i])[0]
b = b.astype(str)
b = b.astype(float)

Теперь перейдём к построению начальной таблицы

In [196]:
def create_table(A, b, c, neg_Q, base):
    #перепишем A, c и Q
    A_new = A.copy()
    c_new = c.copy()
    col_indices = [i for i in range(A.shape[1])]
    row_indices = [j for j in range(A.shape[0])]
    for key, value in base.items():
        row_indices[key] = base[key]
        col_indices.remove(base[key])
        if c[value] != 0:
            neg_Q += b[key] * c[value]
            for i in range(len(c)):
                c_new[i] -= c[value] * A[key][i]
    A_new = np.delete(A_new, row_indices, 1)
    c_new = np.delete(c_new, row_indices, 0)
    return col_indices, row_indices, A_new, c_new, neg_Q

Напишем функции выбора разрешающих столбца и строки и пересчёта таблицы

In [209]:
#Функция выбора разрешающего столбца
def select_col(c):
    return np.argmin(c)

#Функция выбора разрешающей строки
def select_row(selected_col, A, b):
    selected_row =  np.argmin([b[i] / A[i][selected_col] if A[i][selected_col] > 0
                               else np.inf for i in range(b.shape[0])])
    return selected_row

#Переписывание таблицы по выбранному разрешающему элементу
def recount_table(selected_row, selected_col, A, b, c, neg_Q, col_indices, row_indices):
    A_new, b_new, c_new = A.copy(), b.copy(), c.copy()
    
    #Пересчитаем A
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if i == selected_row and j == selected_col:
                A_new[i][j] = 1 / A[selected_row][selected_col]
            elif i == selected_row:
                A_new[i][j] = A[i][j] / A[selected_row][selected_col]
            elif j == selected_col:
                A_new[i][j] = -1 * A[i][j] / A[selected_row][selected_col]
            else:
                A_new[i][j] = A[i][j] - A[selected_row][j] * A[i][selected_col] / A[selected_row][selected_col]

    #Пересчитаем b
    for k in range(len(b)):
        if k == selected_row:
            b_new[k] = b[k] / A[selected_row][selected_col]
        else:
            b_new[k] = b[k] - b[selected_row] * A[k][selected_col] / A[selected_row][selected_col]
    
    #Пересчитаем c
    for l in range(len(c)):
        if l == selected_col:
            c_new[l] = -1 * c[l] / A[selected_row][selected_col]
        else:
            c_new[l] = c[l] - c[selected_col] * A[selected_row][l] / A[selected_row][selected_col]
    
    #Пересчитаем neg_Q
    neg_Q -= b[selected_row] * c[selected_col] / A[selected_row][selected_col]
    
    #Поменяем местами индексы переменных
    old_col_index = col_indices[selected_col]
    col_indices[selected_col] = row_indices[selected_row]
    row_indices[selected_row] = old_col_index
    
    return A_new, b_new, c_new, neg_Q, col_indices, row_indices

In [43]:
"""
A = np.array([
    [15.0, 17.0, 20.0, 9.0],
    [8.0, 17.0, 10.0, 15.0],
    [14.0, 7.0, 15.0, 21.0]
])
b = np.array([48.0, 50.0, 54.0])
c = np.array([-39.0, -45.0, -50.0, -48.0])
neg_Q = 0.0
col_indices, row_indices = [1, 2, 3, 4], [5, 6, 7]
"""

'\nA = np.array([\n    [15.0, 17.0, 20.0, 9.0],\n    [8.0, 17.0, 10.0, 15.0],\n    [14.0, 7.0, 15.0, 21.0]\n])\nb = np.array([48.0, 50.0, 54.0])\nc = np.array([-39.0, -45.0, -50.0, -48.0])\nneg_Q = 0.0\ncol_indices, row_indices = [1, 2, 3, 4], [5, 6, 7]\n'

Напишем функцию поиска базисных переменных и проверки необходимости решения вспомогательной задачи

In [198]:
#Данная функция выдаёт найденный базис и True, если дополнительную задачу решать необходимо, или False иначе
def check_base(A):
    trans_A = A.T.copy()
    diag_matrix = np.diag([1.0 for _ in range(min(trans_A.shape))])
    base = {}

    for i in range(len(diag_matrix)):
        if diag_matrix[i].tolist() in trans_A.tolist():
            temp = np.where(np.all(diag_matrix[i] == trans_A, axis = 1))
            base[i] = temp[0][0]
        else:
            base[i] = None
    if None in base.values():
        return base, True
    else:
        return base, False

In [199]:
#Функция отображения симплекс-таблицы
def simp_show(A, b, c, neg_Q, col_indices, row_indices):
    temp_table = np.hstack((A, np.expand_dims(b, axis = 1)))
    cols = col_indices.copy()
    cols.append('b')
    temp_df = pd.DataFrame(data = temp_table, index = row_indices, columns = cols)
    print(temp_df)
    print('c', c, neg_Q, '\n')
    pass

Напишем функцию выполнения симплекс-метода

In [219]:
def simplex(A, b, c, neg_Q):
    base, aux = check_base(A)
    if aux:
        #решаем дополнительную задачу
        aux = aux_task(A, b)
        if len(aux) == 1:
            pass
        else:
            col_indices, row_indices, A, b, c, neg_Q = aux
            base, _ = check_base(A)
            simp_show(A, b, c, neg_Q, col_indices, row_indices)
    else:
        #создадим и выведем начальную таблицу
        col_indices, row_indices, A, c, neg_Q = create_table(A, b, c, neg_Q, base)
        simp_show(A, b, c, neg_Q, col_indices, row_indices)
    #выполняем шаги симплекс-метода, пока в векторе c есть отрицательные коэффициенты
    while any(i < 0 for i in c):
        #находим разрешающий элемент
        selected_col = select_col(c)
        selected_row = select_row(selected_col, A, b)
        #пересчитываем таблицу
        A, b, c, neg_Q, col_indices, row_indices = recount_table(selected_row, selected_col, A, b, c,
                                                                 neg_Q, col_indices, row_indices)
        #отображаем результат шага
        simp_show(A, b, c, neg_Q, col_indices, row_indices)
    #в конце получаем результаты:
    k = 0
    for i in row_indices:
        print(f"x{i + 1} = {round(b[k], 2)}")
        k += 1
    for j in col_indices:
        print(f"x{j + 1} = 0")
    print(f"Qmin = {-neg_Q}")
    pass

Напишем функцию решения дополнительной задачи

In [220]:
def aux_task(A, b):
    print("решаем дополнительную задачу")
    #перепишем A
    A_new = A.copy()
    fict_base = {}
    for i in range(A.shape[0]):
        fict_base[i] = A.shape[1] + i
        A_new = np.hstack((A_new, np.zeros((3, 1), dtype="float")))
        A_new[i][-1] = 1.0
    
    col_indices = [i for i in range(A_new.shape[1])]
    row_indices = [j for j in range(A_new.shape[0])]
    for key, value in fict_base.items():
        row_indices[key] = fict_base[key]
        col_indices.remove(fict_base[key])
    A_new = np.delete(A_new, row_indices, 1)
    #вычислим c и Q
    c = -1 * np.sum(A_new, axis=0)
    neg_Q = -1 * np.sum(b)
    #выведем начальную таблицу вспомогательной задачи
    simp_show(A_new, b, c, neg_Q, col_indices, row_indices)
    #выполняем шаги симплекс-метода, пока в векторе c есть отрицательные коэффициенты
    b_new = b.copy()
    while any(i < 0 for i in c):
        #находим разрешающий элемент
        selected_col = select_col(c)
        selected_row = select_row(selected_col, A_new, b_new)
        #пересчитываем таблицу
        A_new, b_new, c, neg_Q, col_indices, row_indices = recount_table(selected_row, selected_col, A_new, b_new, c,
                                                                         neg_Q, col_indices, row_indices)
        #отображаем результат шага
        simp_show(A_new, b_new, c, neg_Q, col_indices, row_indices)
        #удаляем столбец, если он соответствует базисной переменной дополнительной задачи
        if col_indices[selected_col] in fict_base.values():
            A_new = np.delete(A_new, selected_col, 1)
            del col_indices[selected_col]
            c = np.delete(c, selected_col)
    if neg_Q > 0.01:
        print('задача не имеет решения')
        return 'F'
    else:
        if not any(ind in fict_base.values() for ind in row_indices):
            c_out = -1 * np.sum(A_new, axis=0)
            neg_Q = -1 * np.sum(b_new)
            print('переходим к основной задаче')
            return col_indices, row_indices, A_new, b_new, c_out, neg_Q
        else:
            while any(ind in fict_base.values() for ind in row_indices):
                for j in range(len(row_indices)):
                    if row_indices[j] in fict_base.values():
                        if A_new[j] == np.zeros(A_new.shape[1]):
                            del row_indices[j]
                            A_new = np.delete(A_new, j, 0)
                        else:
                            selected_col = np.argmax(abs(A_new[j]))
                            selected_row = j
                            A_new, b_new, c, neg_Q, col_indices, row_indices = recount_table(selected_row, selected_col,
                                                                                             A_new, b_new, c, neg_Q,
                                                                                             col_indices, row_indices)
                            #отображаем результат шага
                            simp_show(A_new, b_new, c, neg_Q, col_indices, row_indices)
                            #удаляем столбец, если он соответствует базисной переменной дополнительной задачи
                            A_new = np.delete(A_new, selected_col, 1)
                            col_indices.remove(selected_col)
            if neg_Q > 0:
                print('задача не имеет решения')
                return 'F'
            else:
                c_out = -1 * np.sum(A_new, axis=0)
                neg_Q = -1 * np.sum(b_new)
                print('переходим к основной задаче')
                return col_indices, row_indices, A_new, b_new, c_out, neg_Q

In [221]:
simplex(A, b, c, neg_Q)

решаем дополнительную задачу
     0    1    2    3    4     b
5  3.0  0.0  2.0 -1.0  0.0  12.0
6  1.0 -1.0  1.0  0.0  0.0   5.0
7  1.0  0.0  1.0  0.0  1.0   6.0
c [-5.  1. -4.  1. -1.] -23.0 

          5    1         2         3    4    b
0  0.333333  0.0  0.666667 -0.333333  0.0  4.0
6 -0.333333 -1.0  0.333333  0.333333  0.0  1.0
7 -0.333333  0.0  0.333333  0.333333  1.0  2.0
c [ 1.66666667  1.         -0.66666667 -0.66666667 -1.        ] -3.0 

     1         2         3    7    b
0  0.0  0.666667 -0.333333 -0.0  4.0
6 -1.0  0.333333  0.333333 -0.0  1.0
4  0.0  0.333333  0.333333  1.0  2.0
c [ 1.         -0.33333333 -0.33333333  1.        ] -1.0 

     1    2    6    b
0 -1.0  1.0  1.0  5.0
3 -3.0  1.0  3.0  3.0
4  1.0  0.0 -1.0  1.0
c [-4.44089210e-16  3.33066907e-16  1.00000000e+00] 4.440892098500626e-16 

     4    2    b
0  1.0  1.0  6.0
3  3.0  1.0  6.0
1  1.0  0.0  1.0
c [4.44089210e-16 3.33066907e-16] 8.881784197001252e-16 

переходим к основной задаче
     4    2    b
0  1.0